# OpenAI APIs - Vision

SGLang provides OpenAI-compatible APIs to enable a smooth transition from OpenAI services to self-hosted local models.
A complete reference for the API is available in the [OpenAI API Reference](https://platform.openai.com/docs/guides/vision).
This tutorial covers the vision APIs for vision language models.

SGLang supports various vision language models such as Llama 3.2, LLaVA-OneVision, Qwen2.5-VL, Gemma3 and [more](https://docs.sglang.ai/references/supported_models): 
- [meta-llama/Llama-3.2-11B-Vision-Instruct](https://huggingface.co/meta-llama/Llama-3.2-11B-Vision-Instruct)  
- [lmms-lab/llava-onevision-qwen2-72b-ov-chat](https://huggingface.co/lmms-lab/llava-onevision-qwen2-72b-ov-chat)  
- [Qwen/Qwen2.5-VL-7B-Instruct](https://huggingface.co/Qwen/Qwen2.5-VL-7B-Instruct)
- [google/gemma-3-4b-it](https://huggingface.co/google/gemma-3-4b-it)
- [openbmb/MiniCPM-V](https://huggingface.co/openbmb/MiniCPM-V)
- [deepseek-ai/deepseek-vl2](https://huggingface.co/deepseek-ai/deepseek-vl2)

As an alternative to the OpenAI API, you can also use the [SGLang offline engine](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py).

## Launch A Server

Launch the server in your terminal and wait for it to initialize.

**Remember to add** `--chat-template llama_3_vision` **to specify the [vision chat template](https://docs.sglang.ai/backend/openai_api_vision.html#Chat-Template), otherwise, the server will only support text (images won’t be passed in), which can lead to degraded performance.**

We need to specify `--chat-template` for vision language models because the chat template provided in Hugging Face tokenizer only supports text.

In [1]:
from sglang.test.test_utils import is_in_ci

if is_in_ci():
    from patch import launch_server_cmd
else:
    from sglang.utils import launch_server_cmd

from sglang.utils import wait_for_server, print_highlight, terminate_process

vision_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model-path meta-llama/Llama-3.2-11B-Vision-Instruct \
    --chat-template=llama_3_vision
"""
)

wait_for_server(f"http://localhost:{port}")

[2025-04-01 22:18:10] server_args=ServerArgs(model_path='meta-llama/Llama-3.2-11B-Vision-Instruct', tokenizer_path='meta-llama/Llama-3.2-11B-Vision-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', trust_remote_code=False, dtype='auto', kv_cache_dtype='auto', quantization=None, quantization_param_path=None, context_length=None, device='cuda', served_model_name='meta-llama/Llama-3.2-11B-Vision-Instruct', chat_template='llama_3_vision', completion_template=None, is_embedding=False, revision=None, host='127.0.0.1', port=38824, mem_fraction_static=0.88, max_running_requests=200, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, tp_size=1, stream_interval=1, stream_output=False, random_seed=298739275, constrained_json_whitespace_pattern=None, watchdog_timeout=300, dist_timeout=None, download_dir=None, base_gpu_id=0, gpu_id_step=1, log_level='info', 

[2025-04-01 22:18:12] Use chat template for the OpenAI-compatible API server: llama_3_vision


[2025-04-01 22:18:24 TP0] Overlap scheduler is disabled for multimodal models.
[2025-04-01 22:18:24 TP0] Automatically reduce --mem-fraction-static to 0.792 because this is a multimodal model.
[2025-04-01 22:18:24 TP0] Automatically turn off --chunked-prefill-size for multimodal model.
[2025-04-01 22:18:24 TP0] Init torch distributed begin.


[2025-04-01 22:18:26 TP0] Init torch distributed ends. mem usage=0.00 GB
[2025-04-01 22:18:26 TP0] Load weight begin. avail mem=74.21 GB


[2025-04-01 22:18:27 TP0] Using model weights format ['*.safetensors']
Loading safetensors checkpoint shards:   0% Completed | 0/5 [00:00<?, ?it/s]


...

[2025-04-01 22:18:35 TP0] max_total_num_tokens=20480, chunked_prefill_size=-1, max_prefill_tokens=16384, max_running_requests=200, context_len=131072


[2025-04-01 22:18:36] INFO:     Started server process [304450]
[2025-04-01 22:18:36] INFO:     Waiting for application startup.
[2025-04-01 22:18:36] INFO:     Application startup complete.
[2025-04-01 22:18:36] INFO:     Uvicorn running on http://127.0.0.1:38824 (Press CTRL+C to quit)


[2025-04-01 22:18:36] INFO:     127.0.0.1:51400 - "GET /v1/models HTTP/1.1" 200 OK


[2025-04-01 22:18:37] INFO:     127.0.0.1:51414 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-04-01 22:18:37 TP0] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-01 22:18:39] INFO:     127.0.0.1:51422 - "POST /generate HTTP/1.1" 200 OK
[2025-04-01 22:18:39] The server is fired up and ready to roll!


## Using cURL

Once the server is up, you can send test requests using curl or requests.

In [2]:
import subprocess

curl_command = f"""
curl -s http://localhost:{port}/v1/chat/completions \\
  -d '{{
    "model": "meta-llama/Llama-3.2-11B-Vision-Instruct",
    "messages": [
      {{
        "role": "user",
        "content": [
          {{
            "type": "text",
            "text": "What’s in this image?"
          }},
          {{
            "type": "image_url",
            "image_url": {{
              "url": "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
            }}
          }}
        ]
      }}
    ],
    "max_tokens": 300
  }}'
"""

response = subprocess.check_output(curl_command, shell=True).decode()
print_highlight(response)


response = subprocess.check_output(curl_command, shell=True).decode()
print_highlight(response)

[2025-04-01 22:18:42 TP0] Prefill batch. #new-seq: 1, #new-token: 6462, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-01 22:18:42 TP0] Decode batch. #running-req: 1, #token: 6495, token usage: 0.32, gen throughput (token/s): 5.42, #queue-req: 0, 


[2025-04-01 22:18:43] INFO:     127.0.0.1:51430 - "POST /v1/chat/completions HTTP/1.1" 200 OK


[2025-04-01 22:18:44 TP0] Prefill batch. #new-seq: 1, #new-token: 6462, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-04-01 22:18:44 TP0] Decode batch. #running-req: 1, #token: 6463, token usage: 0.32, gen throughput (token/s): 31.37, #queue-req: 0, 


[2025-04-01 22:18:44 TP0] Decode batch. #running-req: 1, #token: 6503, token usage: 0.32, gen throughput (token/s): 64.92, #queue-req: 0, 


[2025-04-01 22:18:45 TP0] Decode batch. #running-req: 1, #token: 6543, token usage: 0.32, gen throughput (token/s): 64.81, #queue-req: 0, 


[2025-04-01 22:18:46 TP0] Decode batch. #running-req: 1, #token: 0, token usage: 0.00, gen throughput (token/s): 64.28, #queue-req: 0, 
[2025-04-01 22:18:46] INFO:     127.0.0.1:49356 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Using Python Requests

In [3]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "meta-llama/Llama-3.2-11B-Vision-Instruct",
    "messages": [
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "What’s in this image?"},
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
                    },
                },
            ],
        }
    ],
    "max_tokens": 300,
}

response = requests.post(url, json=data)
print_highlight(response.text)

[2025-04-01 22:18:46 TP0] Prefill batch. #new-seq: 1, #new-token: 6462, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-01 22:18:47 TP0] Decode batch. #running-req: 1, #token: 6502, token usage: 0.32, gen throughput (token/s): 32.27, #queue-req: 0, 


[2025-04-01 22:18:47 TP0] Decode batch. #running-req: 1, #token: 6542, token usage: 0.32, gen throughput (token/s): 64.93, #queue-req: 0, 


[2025-04-01 22:18:48 TP0] Decode batch. #running-req: 1, #token: 6582, token usage: 0.32, gen throughput (token/s): 64.65, #queue-req: 0, 


[2025-04-01 22:18:48] INFO:     127.0.0.1:49360 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Using OpenAI Python Client

In [4]:
from openai import OpenAI

client = OpenAI(base_url=f"http://localhost:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-3.2-11B-Vision-Instruct",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What is in this image?",
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true"
                    },
                },
            ],
        }
    ],
    max_tokens=300,
)

print_highlight(response.choices[0].message.content)

[2025-04-01 22:18:49 TP0] Prefill batch. #new-seq: 1, #new-token: 6462, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-01 22:18:49] INFO:     127.0.0.1:49366 - "POST /v1/chat/completions HTTP/1.1" 200 OK


## Multiple-Image Inputs

The server also supports multiple images and interleaved text and images if the model supports it.

In [5]:
from openai import OpenAI

client = OpenAI(base_url=f"http://localhost:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-3.2-11B-Vision-Instruct",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://github.com/sgl-project/sglang/blob/main/test/lang/example_image.png?raw=true",
                    },
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://raw.githubusercontent.com/sgl-project/sglang/main/assets/logo.png",
                    },
                },
                {
                    "type": "text",
                    "text": "I have two very different images. They are not related at all. "
                    "Please describe the first image in one sentence, and then describe the second image in another sentence.",
                },
            ],
        }
    ],
    temperature=0,
)

print_highlight(response.choices[0].message.content)

[2025-04-01 22:19:00] INFO:     127.0.0.1:49376 - "POST /v1/chat/completions HTTP/1.1" 500 Internal Server Error
[2025-04-01 22:19:00] ERROR:    Exception in ASGI application
Traceback (most recent call last):
  File "/public_sglang_ci/runner-l3a-gpu-67/_work/_tool/Python/3.9.21/x64/lib/python3.9/site-packages/urllib3/connection.py", line 174, in _new_conn
    conn = connection.create_connection(
  File "/public_sglang_ci/runner-l3a-gpu-67/_work/_tool/Python/3.9.21/x64/lib/python3.9/site-packages/urllib3/util/connection.py", line 72, in create_connection
    for res in socket.getaddrinfo(host, port, family, socket.SOCK_STREAM):
  File "/public_sglang_ci/runner-l3a-gpu-67/_work/_tool/Python/3.9.21/x64/lib/python3.9/socket.py", line 966, in getaddrinfo
    for res in _socket.getaddrinfo(host, port, family, type, proto, flags):
socket.gaierror: [Errno -3] Temporary failure in name resolution

During handling of the above exception, another exception occurred:

Traceback (most recent call 

[2025-04-01 22:19:01 TP0] Prefill batch. #new-seq: 1, #new-token: 12894, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-04-01 22:19:01 TP0] Decode batch. #running-req: 1, #token: 12897, token usage: 0.63, gen throughput (token/s): 3.00, #queue-req: 0, 


[2025-04-01 22:19:02 TP0] Decode batch. #running-req: 1, #token: 12937, token usage: 0.63, gen throughput (token/s): 64.60, #queue-req: 0, 


[2025-04-01 22:19:03 TP0] Decode batch. #running-req: 1, #token: 12977, token usage: 0.63, gen throughput (token/s): 63.81, #queue-req: 0, 


[2025-04-01 22:19:03] INFO:     127.0.0.1:57532 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [6]:
terminate_process(vision_process)

[2025-04-01 22:19:03] Child process unexpectedly failed with an exit code 9. pid=305048


## Chat Template

As mentioned before, if you do not specify a vision model's `--chat-template`, the server uses Hugging Face's default template, which only supports text.

We list popular vision models with their chat templates:

- [meta-llama/Llama-3.2-Vision](https://huggingface.co/meta-llama/Llama-3.2-11B-Vision-Instruct) uses `llama_3_vision`.
- [Qwen/Qwen2.5-VL-7B-Instruct](https://huggingface.co/Qwen/Qwen2.5-VL-7B-Instruct) uses `qwen2-vl`.
- [google/gemma-3-4b-it](https://huggingface.co/google/gemma-3-4b-it) uses `gemma-it`.
- [openbmb/MiniCPM-V](https://huggingface.co/openbmb/MiniCPM-V) uses `minicpmv`.
- [deepseek-ai/deepseek-vl2](https://huggingface.co/deepseek-ai/deepseek-vl2) uses `deepseek-vl2`.
- [LlaVA-OneVision](https://huggingface.co/lmms-lab/llava-onevision-qwen2-7b-ov) uses `chatml-llava`.
- [LLaVA-NeXT](https://huggingface.co/collections/lmms-lab/llava-next-6623288e2d61edba3ddbf5ff) uses `chatml-llava`.
- [Llama3-LLaVA-NeXT](https://huggingface.co/lmms-lab/llama3-llava-next-8b) uses `llava_llama_3`.
- [LLaVA-v1.5 / 1.6](https://huggingface.co/liuhaotian/llava-v1.6-34b) uses `vicuna_v1.1`.